In [1]:
import numpy as np
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from config import fetch_data

## **CREDIT SCORE IMPACT**

### Credit Score Casual Impact

In [2]:
credit_score_causal_impact_query = "SELECT * FROM gold.analytics_credit_score_impact_analysis ORDER BY avg_credit_score"
credit_score_causal_impact_df = fetch_data(credit_score_causal_impact_query)

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("CLV Impact", "Approval Rate Impact",
                    "Transaction Volume Impact", "Default Rate Impact"),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

# CLV
fig.add_trace(
    go.Scatter(
        x=credit_score_causal_impact_df['credit_score_band'],
        y=credit_score_causal_impact_df['avg_clv'],
        mode='lines+markers',
        name='CLV',
        line=dict(color='#1f77b4', width=3),
        marker=dict(size=8),
        hovertemplate='<b>%{x}</b><br>CLV: $%{y:,.2f}<extra></extra>'
    ),
    row=1, col=1
)

# Approval Rate
fig.add_trace(
    go.Scatter(
        x=credit_score_causal_impact_df['credit_score_band'],
        y=credit_score_causal_impact_df['avg_approval_rate_pct'],
        mode='lines+markers',
        name='Approval',
        line=dict(color='#2ca02c', width=3),
        marker=dict(size=8),
        hovertemplate='<b>%{x}</b><br>Approval Rate: %{y:.1f}%<extra></extra>'
    ),
    row=1, col=2
)

# Transactions
fig.add_trace(
    go.Scatter(
        x=credit_score_causal_impact_df['credit_score_band'],
        y=credit_score_causal_impact_df['avg_transactions'],
        mode='lines+markers',
        name='Transactions',
        line=dict(color='#ff7f0e', width=3),
        marker=dict(size=8),
        hovertemplate='<b>%{x}</b><br>Transactions: %{y:,.0f}<extra></extra>'
    ),
    row=2, col=1
)

# Default Rate
fig.add_trace(
    go.Scatter(
        x=credit_score_causal_impact_df['credit_score_band'],
        y=credit_score_causal_impact_df['default_rate_pct'],
        mode='lines+markers',
        name='Default',
        line=dict(color='#d62728', width=3),
        marker=dict(size=8),
        hovertemplate='<b>%{x}</b><br>Default Rate: %{y:.1f}%<extra></extra>'
    ),
    row=2, col=2
)

# Update axes
for i in range(1,3):
    for j in range(1,3):
        fig.update_xaxes(title_text="Credit Score Band", row=i, col=j)
        
fig.update_yaxes(title_text="Avg CLV ($)", row=1, col=1)
fig.update_yaxes(title_text="Approval Rate (%)", row=1, col=2)
fig.update_yaxes(title_text="Avg Transactions", row=2, col=1)
fig.update_yaxes(title_text="Default Rate (%)", row=2, col=2)

# Layout improvements
fig.update_layout(
    title_text="Causal Impact of Credit Score on Customer Outcomes",
    height=720,
    plot_bgcolor='rgba(245,245,245,1)',
    hovermode='x unified',
    legend=dict(orientation="h", y=-0.1),
)

fig.show()

In [3]:
credit_score_causal_impact_df

,credit_score_band,customer_count,avg_credit_score,stddev_credit_score,avg_transactions,avg_transaction_volume,avg_accounts,avg_balance,avg_approval_rate_pct,default_rate_pct,avg_clv,marginal_effect_transactions,marginal_effect_balance,marginal_effect_approval_pct,clv_change_per_100_credit_points,analyzed_at
0,Poor,12020,439.0,80.47,0.5,802.76,3.01,-99517.58,0.00,46.85,50549.49,NaN,NaN,NaN,NaN,2025-12-27 07:58:01.979666+00:00
1,Fair,3952,624.0,26.07,0.5,812.22,3.01,-92967.20,5.42,46.26,50829.12,0.0,6550.38,5.42,150.55,2025-12-27 07:58:01.979666+00:00
2,Good,2995,705.0,20.16,0.5,821.00,3.02,-106165.62,31.81,45.88,50038.02,0.0,-13198.43,26.39,-984.55,2025-12-27 07:58:01.979666+00:00
3,Very Good,2583,769.0,17.56,0.5,759.04,2.93,-96523.77,49.05,46.77,50405.10,0.0,9641.85,17.24,570.43,2025-12-27 07:58:01.979666+00:00
4,Excellent,28,800.0,0.00,0.3,465.82,3.29,-29985.44,53.57,28.57,52469.39,-0.2,66538.33,4.52,6677.59,2025-12-27 07:58:01.979666+00:00


### Credit Score Marginal Effects

In [4]:
credit_score_marginal_effects_query = "SELECT * FROM gold.analytics_credit_score_impact_analysis ORDER BY avg_credit_score"
credit_score_marginal_effects_df = fetch_data(credit_score_marginal_effects_query)

hover_text = [
    f"<b>{band}</b><br>CLV Change: ${value:,.2f} per 100 pts"
    for band, value in zip(
        credit_score_marginal_effects_df['credit_score_band'],
        credit_score_marginal_effects_df['clv_change_per_100_credit_points']
    )
]

fig = go.Figure()

# Marginal CLV effect
fig.add_trace(go.Bar(
    name='CLV Change per 100 pts',
    x=credit_score_marginal_effects_df['credit_score_band'],
    y=credit_score_marginal_effects_df['clv_change_per_100_credit_points'],
    marker_color='#636EFA',
    text=credit_score_marginal_effects_df['clv_change_per_100_credit_points'],
    texttemplate='$%{text:,.0f}',
    textposition='outside',
    hovertext=hover_text,
     hoverinfo="text"
))

fig.update_layout(
    title="Marginal Effect: CLV Change per 100 Credit Score Points",
    xaxis_title="Credit Score Band",
    yaxis_title="CLV Change ($)",
    height=500,
    plot_bgcolor='rgba(245,245,245,1)',
    yaxis=dict(showgrid=True, gridcolor='lightgray', zeroline=True, zerolinecolor='black')
)
fig.show()

In [5]:
credit_score_marginal_effects_df

,credit_score_band,customer_count,avg_credit_score,stddev_credit_score,avg_transactions,avg_transaction_volume,avg_accounts,avg_balance,avg_approval_rate_pct,default_rate_pct,avg_clv,marginal_effect_transactions,marginal_effect_balance,marginal_effect_approval_pct,clv_change_per_100_credit_points,analyzed_at
0,Poor,12020,439.0,80.47,0.5,802.76,3.01,-99517.58,0.00,46.85,50549.49,NaN,NaN,NaN,NaN,2025-12-27 07:58:01.979666+00:00
1,Fair,3952,624.0,26.07,0.5,812.22,3.01,-92967.20,5.42,46.26,50829.12,0.0,6550.38,5.42,150.55,2025-12-27 07:58:01.979666+00:00
2,Good,2995,705.0,20.16,0.5,821.00,3.02,-106165.62,31.81,45.88,50038.02,0.0,-13198.43,26.39,-984.55,2025-12-27 07:58:01.979666+00:00
3,Very Good,2583,769.0,17.56,0.5,759.04,2.93,-96523.77,49.05,46.77,50405.10,0.0,9641.85,17.24,570.43,2025-12-27 07:58:01.979666+00:00
4,Excellent,28,800.0,0.00,0.3,465.82,3.29,-29985.44,53.57,28.57,52469.39,-0.2,66538.33,4.52,6677.59,2025-12-27 07:58:01.979666+00:00


## **CAMPAIGN CAUSAL IMPACT (DiD)**

### Campaign Did Analysis

In [6]:
campaign_did_analysis_query = "SELECT * FROM gold.analytics_campaign_causal_impact ORDER BY ABS(did_estimate_transactions) DESC LIMIT 15"
campaign_did_analysis_df = fetch_data(campaign_did_analysis_query)


# Create hover text
hover_before = [
    f"<b>{campaign}</b><br>Before: {val:.1f}" 
    for campaign, val in zip(
        campaign_did_analysis_df['campaign_name'], 
        campaign_did_analysis_df['avg_transactions_before']
    )
]

hover_after = [
    f"<b>{campaign}</b><br>After: {val:.1f}" 
    for campaign, val in zip(
        campaign_did_analysis_df['campaign_name'], 
        campaign_did_analysis_df['avg_transactions_after']
    )
]

hover_did = [
    f"<b>{campaign}</b><br>DiD Effect: {val:.2f}" 
    for campaign, val in zip(
        campaign_did_analysis_df['campaign_name'], 
        campaign_did_analysis_df['did_estimate_transactions']
    )
]

fig = go.Figure()

# Before Campaign
fig.add_trace(go.Bar(
    name='Before Campaign',
    x=campaign_did_analysis_df['campaign_name'],
    y=campaign_did_analysis_df['avg_transactions_before'],
    marker_color='lightgray',
    hovertext=hover_before,
    hoverinfo="text"
))

# After Campaign
fig.add_trace(go.Bar(
    name='After Campaign',
    x=campaign_did_analysis_df['campaign_name'],
    y=campaign_did_analysis_df['avg_transactions_after'],
    marker_color='#00CC96',
    hovertext=hover_after,
    hoverinfo="text"
))

# DiD Estimate markers
fig.add_trace(go.Scatter(
    name='DiD Estimate',
    x=campaign_did_analysis_df['campaign_name'],
    y=campaign_did_analysis_df['avg_transactions_after'],
    mode='markers',
    marker=dict(
        size=abs(campaign_did_analysis_df['did_estimate_transactions']) * 3,
        color=campaign_did_analysis_df['did_estimate_transactions'],
        colorscale='RdYlGn',
        showscale=True,
        colorbar=dict(title="DiD Effect")
    ),
    hovertext=hover_did,
    hoverinfo="text"
))

fig.update_layout(
    title="Campaign Causal Impact: Before vs After (Difference-in-Differences, Top 15)",
    xaxis_title="Campaign",
    yaxis_title="Avg Transactions",
    barmode='group',
    height=600,
    plot_bgcolor='rgba(245,245,245,1)',
    xaxis_tickangle=-45,
    yaxis=dict(showgrid=True, gridcolor='lightgray')
)

fig.show()

In [7]:
campaign_did_analysis_df

,campaign_id,campaign_name,target_segment,start_date,end_date,conversions,avg_transactions_before,avg_transactions_after,transaction_change,transaction_change_pct,did_estimate_transactions,balance_change,balance_change_pct,clv_change,causal_interpretation,significance,analyzed_at
0,100,Cross-group systemic model,Business,2025-09-28,2025-11-11,3310,0.05,0.04,-0.01,-11.42,0.0,164.61,-0.18,0.0,No Impact,Not Significant,2025-12-27 07:58:01.295598+00:00
1,105,Ameliorated zero tolerance secured line,Premium,2025-11-11,2025-12-15,501,0.05,0.04,0.00,-9.92,0.0,118.47,-0.11,0.0,No Impact,Not Significant,2025-12-27 07:58:01.295598+00:00
2,240,Implemented static application,Premium,2025-11-03,2025-12-05,2443,0.05,0.04,0.00,-9.92,0.0,118.47,-0.11,0.0,No Impact,Not Significant,2025-12-27 07:58:01.295598+00:00


### Campaign Impact Heatmap

In [8]:
campaign_impact_heatmap_query = "SELECT * FROM gold.analytics_campaign_causal_impact ORDER BY ABS(did_estimate_transactions) DESC"
campaign_impact_heatmap_df = fetch_data(campaign_impact_heatmap_query)

# Metrics
metrics = ['transaction_change_pct', 'balance_change_pct', 'clv_change']
y_labels = ['Transaction Change %', 'Balance Change %', 'CLV Change ($)']

# Prepare z-data and hovertext
z_data = []
hover_text = []

for metric, label in zip(metrics, y_labels):
    row_values = campaign_impact_heatmap_df[metric].values
    z_data.append(row_values)
    hover_text.append([
        f"<b>{label}</b><br>Campaign: {campaign}<br>Change: {val:,.2f}" 
        for campaign, val in zip(campaign_impact_heatmap_df['campaign_name'], row_values)
    ])

fig = go.Figure(data=go.Heatmap(
    z=z_data,
    x=campaign_impact_heatmap_df['campaign_name'],
    y=y_labels,
    colorscale='RdYlGn',
    text=z_data,
    texttemplate='%{text:.1f}',
    textfont={"size": 9},
    hovertext=hover_text,
    hoverinfo="text",
    colorbar=dict(title="Impact")
))

fig.update_layout(
    title="Campaign Causal Impact Heatmap (Top 20)",
    height=500,
    xaxis_title="Campaign",
    yaxis_title="Impact Metric",
    xaxis_tickangle=-45,
    plot_bgcolor='rgba(245,245,245,1)'
)

fig.show()

In [9]:
campaign_impact_heatmap_df

,campaign_id,campaign_name,target_segment,start_date,end_date,conversions,avg_transactions_before,avg_transactions_after,transaction_change,transaction_change_pct,did_estimate_transactions,balance_change,balance_change_pct,clv_change,causal_interpretation,significance,analyzed_at
0,100,Cross-group systemic model,Business,2025-09-28,2025-11-11,3310,0.05,0.04,-0.01,-11.42,0.0,164.61,-0.18,0.0,No Impact,Not Significant,2025-12-27 07:58:01.295598+00:00
1,105,Ameliorated zero tolerance secured line,Premium,2025-11-11,2025-12-15,501,0.05,0.04,0.00,-9.92,0.0,118.47,-0.11,0.0,No Impact,Not Significant,2025-12-27 07:58:01.295598+00:00
2,240,Implemented static application,Premium,2025-11-03,2025-12-05,2443,0.05,0.04,0.00,-9.92,0.0,118.47,-0.11,0.0,No Impact,Not Significant,2025-12-27 07:58:01.295598+00:00


## **INTEREST RATE ELASTICITY**

### Interest Rate Elasticity

In [10]:
interest_rate_elasticity_query = "SELECT * FROM gold.analytics_interest_rate_behavior_impact ORDER BY avg_interest_rate"
interest_rate_elasticity_df = fetch_data(interest_rate_elasticity_query)


fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=("Balance Elasticity",
                    "Churn & Late Payment Elasticity"),
    vertical_spacing=0.12
)

# Create hovertext
hover_balance = [
    f"Rate: {rate:.2f}%<br>Elasticity: {elasticity:.2f}" 
    for rate, elasticity in zip(
        interest_rate_elasticity_df['avg_interest_rate'],
        interest_rate_elasticity_df['balance_elasticity']
    )
]

hover_churn = [
    f"Rate: {rate:.2f}%<br>Churn Elasticity: {elasticity:.2f}%" 
    for rate, elasticity in zip(
        interest_rate_elasticity_df['avg_interest_rate'],
        interest_rate_elasticity_df['churn_elasticity_pct']
    )
]

hover_late = [
    f"Rate: {rate:.2f}%<br>Late Payment Elasticity: {elasticity:.2f}%" 
    for rate, elasticity in zip(
        interest_rate_elasticity_df['avg_interest_rate'],
        interest_rate_elasticity_df['late_payment_elasticity_pct']
    )
]

# Balance elasticity
fig.add_trace(
    go.Scatter(
        x=interest_rate_elasticity_df['avg_interest_rate'],
        y=interest_rate_elasticity_df['balance_elasticity'],
        mode='lines+markers',
        name='Balance Elasticity',
        line=dict(color='#636EFA', width=3),
        marker=dict(size=8),
        hovertext=hover_balance,
        hoverinfo="text"
    ),
    row=1, col=1
)

# Churn elasticity
fig.add_trace(
    go.Scatter(
        x=interest_rate_elasticity_df['avg_interest_rate'],
        y=interest_rate_elasticity_df['churn_elasticity_pct'],
        mode='lines+markers',
        name='Churn Elasticity',
        line=dict(color='#EF553B', width=3),
        marker=dict(size=8),
        hovertext=hover_churn,
        hoverinfo="text"
    ),
    row=2, col=1
)

# Late payment elasticity
fig.add_trace(
    go.Scatter(
        x=interest_rate_elasticity_df['avg_interest_rate'],
        y=interest_rate_elasticity_df['late_payment_elasticity_pct'],
        mode='lines+markers',
        name='Late Payment Elasticity',
        line=dict(color='#FFA15A', width=3),
        marker=dict(size=8),
        hovertext=hover_late,
        hoverinfo="text"
    ),
    row=2, col=1
)

# Add zero line
for row in [1, 2]:
    fig.add_hline(y=0, line_dash="dash", line_color="gray",
                  opacity=0.5, row=row, col=1)

# Axes
fig.update_xaxes(title_text="Interest Rate (%)", row=2, col=1)
fig.update_yaxes(title_text="Balance Elasticity", row=1, col=1)
fig.update_yaxes(title_text="Elasticity (%)", row=2, col=1)

# Layout
fig.update_layout(
    title="Causal Effect of Interest Rate Changes on Customer Behavior",
    height=700,
    plot_bgcolor='rgba(245,245,245,1)',
    hovermode='x unified'
)

fig.show()

In [11]:
interest_rate_elasticity_df

,rate_category,account_count,avg_interest_rate,stddev_rate,avg_balance,avg_transactions,avg_transaction_size,avg_late_payment_rate_pct,past_due_rate_pct,avg_churn_risk_pct,balance_elasticity,late_payment_elasticity_pct,churn_elasticity_pct,balance_impact_interpretation,analyzed_at
0,Low Rate,32321,2.80,1.37,-12161.28,0.1,116.82,2.93,0.0,50.09,NaN,NaN,NaN,Inconclusive,2025-12-27 07:58:04.808749+00:00
1,Medium Rate,13032,6.75,1.25,-251472.49,0.1,120.30,14.50,0.0,50.15,-60541.16,2.93,0.02,High Negative Impact on Balances,2025-12-27 07:58:04.808749+00:00
2,High Rate,12850,16.49,1.50,-8869.09,0.1,123.10,0.00,0.0,49.89,24907.83,-1.49,-0.03,Positive or No Impact,2025-12-27 07:58:04.808749+00:00


### Interest Rate Impact Summary

In [12]:
interest_rate_impact_summary_query = "SELECT * FROM gold.analytics_interest_rate_behavior_impact ORDER BY avg_interest_rate"
interest_rate_impact_summary_df = fetch_data(interest_rate_impact_summary_query)

# Hovertext
hover_balance = [
    f"<b>{cat}</b><br>Avg Balance: ${val:,.2f}" 
    for cat, val in zip(
        interest_rate_impact_summary_df['rate_category'], 
        interest_rate_impact_summary_df['avg_balance']
    )
]

hover_late = [
    f"<b>{cat}</b><br>Late Payment Rate: {val:.1f}%" 
    for cat, val in zip(
        interest_rate_impact_summary_df['rate_category'], 
        interest_rate_impact_summary_df['avg_late_payment_rate_pct']
    )
]

fig = go.Figure()

# Avg Balance (bar)
fig.add_trace(go.Bar(
    name='Avg Balance',
    x=interest_rate_impact_summary_df['rate_category'],
    y=interest_rate_impact_summary_df['avg_balance'],
    marker_color='#636EFA',
    yaxis='y',
    hovertext=hover_balance,
    hoverinfo="text"
))

# Late Payment Rate (line)
fig.add_trace(go.Scatter(
    name='Late Payment Rate',
    x=interest_rate_impact_summary_df['rate_category'],
    y=interest_rate_impact_summary_df['avg_late_payment_rate_pct'],
    mode='lines+markers',
    marker=dict(size=10, color='#EF553B'),
    line=dict(width=3),
    yaxis='y2',
    hovertext=hover_late,
    hoverinfo="text"
))

# Layout
fig.update_layout(
    title="Interest Rate Impact: Balance vs Late Payment Rate",
    xaxis_title="Rate Category",
    yaxis=dict(
        title="Avg Balance ($)", 
        side='left',
        showgrid=True,
        gridcolor='lightgray'
    ),
    yaxis2=dict(
        title="Late Payment Rate (%)",
        side='right', 
        overlaying='y',
        showgrid=False
    ),
    height=500,
    plot_bgcolor='rgba(245,245,245,1)',
    hovermode='x unified'
)

fig.show()

In [13]:
interest_rate_impact_summary_df

,rate_category,account_count,avg_interest_rate,stddev_rate,avg_balance,avg_transactions,avg_transaction_size,avg_late_payment_rate_pct,past_due_rate_pct,avg_churn_risk_pct,balance_elasticity,late_payment_elasticity_pct,churn_elasticity_pct,balance_impact_interpretation,analyzed_at
0,Low Rate,32321,2.80,1.37,-12161.28,0.1,116.82,2.93,0.0,50.09,NaN,NaN,NaN,Inconclusive,2025-12-27 07:58:04.808749+00:00
1,Medium Rate,13032,6.75,1.25,-251472.49,0.1,120.30,14.50,0.0,50.15,-60541.16,2.93,0.02,High Negative Impact on Balances,2025-12-27 07:58:04.808749+00:00
2,High Rate,12850,16.49,1.50,-8869.09,0.1,123.10,0.00,0.0,49.89,24907.83,-1.49,-0.03,Positive or No Impact,2025-12-27 07:58:04.808749+00:00


## **BRANCH PROXIMITY IMPACT**

In [14]:
branch_proximity_impact_query = """
    SELECT * FROM gold.analytics_branch_proximity_impact 
    ORDER BY CASE proximity_category 
    WHEN 'Same City' THEN 1 
    WHEN 'Same State' THEN 2 
    ELSE 3 
    END
"""
branch_proximity_impact_df = fetch_data(branch_proximity_impact_query)

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Branch Usage by Proximity",
                    "Churn Risk by Proximity")
)

segments = branch_proximity_impact_df['customer_segment'].unique()

# Branch usage
for segment in segments:
    segment_data = branch_proximity_impact_df[branch_proximity_impact_df['customer_segment'] == segment]
    hovertext_usage = [
        f"<b>{proximity}</b> - {segment}<br>Usage: {usage:.1f}%"
        for proximity, usage in zip(segment_data['proximity_category'], segment_data['branch_usage_pct'])
    ]
    fig.add_trace(
        go.Bar(
            name=segment,
            x=segment_data['proximity_category'],
            y=segment_data['branch_usage_pct'],
            hovertext=hovertext_usage,
            hoverinfo="text"
        ),
        row=1, col=1
    )

# Churn risk
for segment in segments:
    segment_data = branch_proximity_impact_df[branch_proximity_impact_df['customer_segment'] == segment]
    hovertext_churn = [
        f"<b>{proximity}</b> - {segment}<br>Churn Risk: {risk:.1f}%"
        for proximity, risk in zip(segment_data['proximity_category'], segment_data['avg_churn_risk_pct'])
    ]
    fig.add_trace(
        go.Bar(
            name=segment,
            x=segment_data['proximity_category'],
            y=segment_data['avg_churn_risk_pct'],
            hovertext=hovertext_churn,
            hoverinfo="text",
            showlegend=False  # Only show legend once
        ),
        row=1, col=2
    )

# Axes
fig.update_xaxes(title_text="Proximity", row=1, col=1)
fig.update_xaxes(title_text="Proximity", row=1, col=2)
fig.update_yaxes(title_text="Branch Usage (%)", row=1, col=1)
fig.update_yaxes(title_text="Churn Risk (%)", row=1, col=2)

# Layout
fig.update_layout(
    title="Causal Impact of Branch Proximity on Customer Behavior",
    height=500,
    barmode='group',
    plot_bgcolor='rgba(245,245,245,1)',
)

fig.show()

In [15]:
branch_proximity_impact_df

,proximity_category,customer_segment,customer_count,avg_branch_transactions,avg_mobile_transactions,branch_usage_pct,avg_balance,avg_total_transactions,avg_tenure_months,avg_churn_risk_pct,branch_usage_effect,churn_risk_effect,causal_interpretation,analyzed_at
0,Same State,Affluent,5382,0.05,0.05,19.12,-116923.49,0.24,93.7,50.31,0.0,0.0,Moderate Proximity Effect,2025-12-27 07:58:01.018605+00:00
1,Same State,Business,5383,0.05,0.05,21.26,-104388.61,0.25,96.5,49.78,0.0,0.0,Moderate Proximity Effect,2025-12-27 07:58:01.018605+00:00
2,Same State,Mass Market,5375,0.05,0.05,19.77,-108140.33,0.27,94.8,50.27,0.0,0.0,Moderate Proximity Effect,2025-12-27 07:58:01.018605+00:00
3,Same State,Premium,5429,0.04,0.05,17.96,-115977.96,0.24,95.3,50.15,0.0,0.0,Moderate Proximity Effect,2025-12-27 07:58:01.018605+00:00


### Proximity Casual Effect Waterfall

In [16]:
proximity_causal_effect_waterfall_query = """
    SELECT proximity_category, customer_segment,
        AVG(branch_usage_effect) as avg_usage_effect,
        AVG(churn_risk_effect) as avg_churn_effect
    FROM gold.analytics_branch_proximity_impact
    GROUP BY proximity_category, customer_segment
    ORDER BY proximity_category, customer_segment
"""

proximity_causal_effect_waterfall_df = fetch_data(proximity_causal_effect_waterfall_query)

# Aggregate by proximity
prox_agg = proximity_causal_effect_waterfall_df.groupby('proximity_category').agg({
    'avg_usage_effect': 'mean',
    'avg_churn_effect': 'mean'
}).reset_index()

fig = go.Figure(go.Waterfall(
    name="Causal Effect",
    orientation="v",
    measure=["relative", "relative", "total"],
    x=prox_agg['proximity_category'].tolist() + ["Total Effect"],
    y=prox_agg['avg_usage_effect'].tolist() + [0],
    text=[f"{v:+.2f}" for v in prox_agg['avg_usage_effect'].tolist()] +
    [f"{prox_agg['avg_usage_effect'].sum():.2f}"],
    textposition="outside",
    connector={"line": {"color": "rgb(63, 63, 63)"}},
    increasing={"marker": {"color": "#00CC96"}},
    decreasing={"marker": {"color": "#EF553B"}},
    totals={"marker": {"color": "#636EFA"}}
))

fig.update_layout(
    title="Cumulative Causal Effect of Branch Proximity on Usage",
    yaxis_title="Branch Usage Effect",
    height=500
)

In [17]:
proximity_causal_effect_waterfall_df

,proximity_category,customer_segment,avg_usage_effect,avg_churn_effect
0,Same State,Affluent,0.0,0.0
1,Same State,Business,0.0,0.0
2,Same State,Mass Market,0.0,0.0
3,Same State,Premium,0.0,0.0


## **PRODUCT ADOPTION IMPACT**

### Product Adoption Casual Effect

In [18]:
product_adoption_causal_effect_query = "SELECT * FROM gold.analytics_product_adoption_impact ORDER BY customer_segment, product_count"
product_adoption_causal_effect_df = fetch_data(product_adoption_causal_effect_query)

fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=("CLV Effect by Product Count",
                    "Churn Risk Effect by Product Count"),
    vertical_spacing=0.15
)

segments = product_adoption_causal_effect_df['customer_segment'].unique()

for segment in segments:
    segment_data = product_adoption_causal_effect_df[
        product_adoption_causal_effect_df['customer_segment'] == segment
    ]
    
    # Hovertext
    hover_clv = [
        f"<b>{segment}</b><br>Products: {count}<br>CLV Effect: ${val:,.2f}" 
        for count, val in zip(segment_data['product_count'], segment_data['clv_effect_vs_baseline'])
    ]
    hover_churn = [
        f"<b>{segment}</b><br>Products: {count}<br>Churn Effect: {val:.4f}" 
        for count, val in zip(segment_data['product_count'], segment_data['churn_risk_effect_vs_baseline'])
    ]

    # CLV Effect
    fig.add_trace(
        go.Scatter(
            x=segment_data['product_count'],
            y=segment_data['clv_effect_vs_baseline'],
            mode='lines+markers',
            name=segment,
            line=dict(width=3),
            marker=dict(size=8),
            hovertext=hover_clv,
            hoverinfo="text"
        ),
        row=1, col=1
    )

    # Churn Risk Effect
    fig.add_trace(
        go.Scatter(
            x=segment_data['product_count'],
            y=segment_data['churn_risk_effect_vs_baseline'],
            mode='lines+markers',
            name=segment,
            line=dict(width=3),
            marker=dict(size=8),
            hovertext=hover_churn,
            hoverinfo="text",
            showlegend=False  # Avoid duplicate legend
        ),
        row=2, col=1
    )

# Add zero lines
for row in [1, 2]:
    fig.add_hline(y=0, line_dash="dash", line_color="gray",
                  opacity=0.5, row=row, col=1)

# Axes
fig.update_xaxes(title_text="Number of Products", row=2, col=1)
fig.update_yaxes(title_text="CLV Effect ($)", row=1, col=1)
fig.update_yaxes(title_text="Churn Risk Effect", row=2, col=1)

# Layout
fig.update_layout(
    title="Causal Effect of Product Adoption on Customer Value & Retention",
    height=800,
    plot_bgcolor='rgba(245,245,245,1)',
    hovermode='x unified'
)

fig.show()

In [19]:
product_adoption_causal_effect_df

,product_count,customer_segment,customer_count,avg_clv,avg_transactions,avg_volume,avg_churn_risk_pct,clv_effect_vs_baseline,transaction_effect_vs_baseline,churn_risk_effect_vs_baseline,marginal_clv_effect,adoption_level,analyzed_at
0,0,Affluent,661,51466.49,0.0,0.00,51.83,0.00,0.00,0.0000,NaN,None,2025-12-27 07:58:05.790919+00:00
1,1,Affluent,1740,50109.76,0.1,105.57,48.59,-1356.72,0.06,-0.0324,-1356.72,Baseline (1 Product),2025-12-27 07:58:05.790919+00:00
2,2,Affluent,1478,50338.53,0.1,181.53,50.64,-1127.95,0.12,-0.0119,228.77,Cross-Sell Effect (2 Products),2025-12-27 07:58:05.790919+00:00
3,3,Affluent,1002,51625.65,0.2,293.57,50.77,159.16,0.21,-0.0107,1287.12,Multi-Product Effect (3+ Products),2025-12-27 07:58:05.790919+00:00
4,4,Affluent,416,50666.89,0.3,384.29,51.51,-799.59,0.27,-0.0032,-958.76,Multi-Product Effect (3+ Products),2025-12-27 07:58:05.790919+00:00
5,5,Affluent,86,48281.85,0.3,413.94,49.49,-3184.64,0.26,-0.0235,-2385.05,Multi-Product Effect (3+ Products),2025-12-27 07:58:05.790919+00:00
6,0,Business,715,50876.81,0.0,0.00,50.68,0.00,0.00,0.0000,NaN,None,2025-12-27 07:58:05.790919+00:00
7,1,Business,1784,50051.88,0.1,109.91,50.33,-824.93,0.06,-0.0035,-824.93,Baseline (1 Product),2025-12-27 07:58:05.790919+00:00
8,2,Business,1434,51122.58,0.1,226.24,49.42,245.77,0.15,-0.0125,1070.70,Cross-Sell Effect (2 Products),2025-12-27 07:58:05.790919+00:00
9,3,Business,955,51371.86,0.2,337.00,49.38,495.05,0.22,-0.0129,249.28,Multi-Product Effect (3+ Products),2025-12-27 07:58:05.790919+00:00


### Product Marginal Effect Bar 

In [20]:
product_marginal_effects_bar_query = """
    SELECT customer_segment, product_count, marginal_clv_effect
    FROM gold.analytics_product_adoption_impact
    WHERE marginal_clv_effect IS NOT NULL
    ORDER BY customer_segment, product_count
"""

product_marginal_effects_bar_df = fetch_data(product_marginal_effects_bar_query)

fig = go.Figure()

for segment in product_marginal_effects_bar_df['customer_segment'].unique():
    segment_data = product_marginal_effects_bar_df[
        product_marginal_effects_bar_df['customer_segment'] == segment
    ]
    
    # Hovertext
    hovertext = [
        f"<b>{segment}</b><br>From {prod} products<br>Marginal CLV: ${val:,.2f}"
        for prod, val in zip(segment_data['product_count'], segment_data['marginal_clv_effect'])
    ]
    
    fig.add_trace(go.Bar(
        name=segment,
        x=segment_data['product_count'].astype(str),
        y=segment_data['marginal_clv_effect'],
        text=segment_data['marginal_clv_effect'],
        texttemplate='$%{text:,.0f}',
        textposition='outside',
        hovertext=hovertext,
        hoverinfo="text"
    ))

fig.update_layout(
    title="Marginal CLV Effect per Additional Product",
    xaxis_title="Number of Products",
    yaxis_title="Marginal CLV Effect ($)",
    barmode='group',
    height=500,
    plot_bgcolor='rgba(245,245,245,1)',
    yaxis=dict(showgrid=True, gridcolor='lightgray')
)

fig.show()

## **CHANNEL MIGRATION IMPACT**

### Channel Migration Satisfaction

In [21]:
channel_migration_satisfaction_query = "SELECT * FROM gold.analytics_channel_migration_impact ORDER BY customer_segment, dominant_channel"
channel_migration_satisfaction_df = fetch_data(channel_migration_satisfaction_query)

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Satisfaction by Channel", "Churn Risk by Channel")
)

segments = channel_migration_satisfaction_df['customer_segment'].unique()

for segment in segments:
    segment_data = channel_migration_satisfaction_df[
        channel_migration_satisfaction_df['customer_segment'] == segment
    ]
    
    # Hovertext
    hover_satisfaction = [
        f"<b>{segment}</b><br>{channel}<br>Satisfaction: {val:.2f}"
        for channel, val in zip(segment_data['dominant_channel'], segment_data['avg_satisfaction_rating'])
    ]
    hover_churn = [
        f"<b>{segment}</b><br>{channel}<br>Churn Risk: {val:.1f}%"
        for channel, val in zip(segment_data['dominant_channel'], segment_data['avg_churn_risk_pct'])
    ]
    
    # Satisfaction
    fig.add_trace(
        go.Bar(
            name=segment,
            x=segment_data['dominant_channel'],
            y=segment_data['avg_satisfaction_rating'],
            hovertext=hover_satisfaction,
            hoverinfo="text"
        ),
        row=1, col=1
    )
    
    # Churn Risk
    fig.add_trace(
        go.Bar(
            name=segment,
            x=segment_data['dominant_channel'],
            y=segment_data['avg_churn_risk_pct'],
            hovertext=hover_churn,
            hoverinfo="text",
            showlegend=False
        ),
        row=1, col=2
    )

# Axes
fig.update_xaxes(title_text="Channel", row=1, col=1)
fig.update_xaxes(title_text="Channel", row=1, col=2)
fig.update_yaxes(title_text="Avg Satisfaction", row=1, col=1)
fig.update_yaxes(title_text="Churn Risk (%)", row=1, col=2)

# Layout
fig.update_layout(
    title="Causal Impact of Channel Choice on Customer Outcomes",
    height=500,
    barmode='group',
    plot_bgcolor='rgba(245,245,245,1)'
)

fig.show()

In [22]:
channel_migration_satisfaction_df

,dominant_channel,customer_segment,channel_strategy,customer_count,digital_adoption_pct,avg_satisfaction_rating,avg_clv,avg_churn_risk_pct,avg_service_interactions,satisfaction_effect_vs_branch,churn_effect_vs_branch,multichannel_clv_premium,channel_type,analyzed_at
0,ATM,Affluent,Dual-Channel,62,23.66,1.02,55153.76,54.24,0.5,1.02,0.0658,4187.77,Hybrid Channel,2025-12-27 07:58:01.337934+00:00
1,ATM,Affluent,Dual-Channel,62,23.66,1.02,55153.76,54.24,0.5,0.18,0.0231,4187.77,Hybrid Channel,2025-12-27 07:58:01.337934+00:00
2,ATM,Affluent,Dual-Channel,62,23.66,1.02,55153.76,54.24,0.5,0.06,-0.0220,4187.77,Hybrid Channel,2025-12-27 07:58:01.337934+00:00
3,ATM,Affluent,Single-Channel,189,0.00,0.82,49050.25,51.88,0.4,0.82,0.0422,4187.77,Hybrid Channel,2025-12-27 07:58:01.337934+00:00
4,ATM,Affluent,Single-Channel,189,0.00,0.82,49050.25,51.88,0.4,-0.13,-0.0456,4187.77,Hybrid Channel,2025-12-27 07:58:01.337934+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,POS,Premium,Dual-Channel,3,33.33,0.67,28730.33,37.33,0.3,-0.27,-0.1135,-7747.27,Hybrid Channel,2025-12-27 07:58:01.337934+00:00
170,POS,Premium,Dual-Channel,3,33.33,0.67,28730.33,37.33,0.3,0.47,-0.1707,-7747.27,Hybrid Channel,2025-12-27 07:58:01.337934+00:00
171,None,Premium,Multi-Channel,4328,NaN,0.87,50213.12,50.24,0.5,-0.07,0.0155,-7747.27,Hybrid Channel,2025-12-27 07:58:01.337934+00:00
172,None,Premium,Multi-Channel,4328,NaN,0.87,50213.12,50.24,0.5,0.67,-0.0416,-7747.27,Hybrid Channel,2025-12-27 07:58:01.337934+00:00


### Multichannel Premium Waterfall

In [23]:
multichannel_premium_waterfall_query = """
    SELECT channel_strategy, AVG(multichannel_clv_premium) as avg_premium
    FROM gold.analytics_channel_migration_impact
    WHERE multichannel_clv_premium IS NOT NULL
    GROUP BY channel_strategy
    ORDER BY CASE channel_strategy
        WHEN 'Single-Channel' THEN 1
        WHEN 'Dual-Channel' THEN 2
        WHEN 'Multi-Channel' THEN 3
    END
"""
multichannel_premium_waterfall_df = fetch_data(multichannel_premium_waterfall_query)

# Waterfall values
x_labels = multichannel_premium_waterfall_df['channel_strategy'].tolist() + ["Total Premium"]
y_values = multichannel_premium_waterfall_df['avg_premium'].tolist()
total_value = sum(y_values)
y_values.append(total_value)

# Hovertext
hover_text = [
    f"<b>{strategy}</b><br>CLV Premium: ${val:,.2f}" 
    for strategy, val in zip(multichannel_premium_waterfall_df['channel_strategy'], y_values[:-1])
] + [f"<b>Total Premium</b><br>CLV Premium: ${total_value:,.2f}"]

fig = go.Figure(go.Waterfall(
    name="CLV Premium",
    orientation="v",
    measure=["relative"]*len(multichannel_premium_waterfall_df) + ["total"],
    x=x_labels,
    y=y_values,
    text=[f"${v:,.0f}" for v in y_values],
    textposition="outside",
    hovertext=hover_text,
    hoverinfo="text",
    connector={"line": {"color": "rgb(63, 63, 63)"}},
    increasing={"marker": {"color": "#00CC96"}},
    totals={"marker": {"color": "#636EFA"}}
))

fig.update_layout(
    title="Multi-Channel CLV Premium Effect",
    yaxis_title="CLV Premium ($)",
    height=500,
    plot_bgcolor='rgba(245,245,245,1)'
)

fig.show()

In [24]:
multichannel_premium_waterfall_df

,channel_strategy,avg_premium
0,Single-Channel,-1065.87
1,Dual-Channel,-1065.87
2,Multi-Channel,-986.55


## **PAYMENT TIMELINESS IMPACT**

### Payment Behavior Credit Impact

In [25]:
payment_behavior_credit_impact_query = "SELECT * FROM gold.analytics_payment_timeliness_impact ORDER BY customer_segment, payment_behavior_category"
payment_behavior_credit_impact_df = fetch_data(payment_behavior_credit_impact_query)

fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=("Credit Score by Payment Behavior",
                    "Credit Utilization by Payment Behavior"),
    vertical_spacing=0.15
)

segments = payment_behavior_credit_impact_df['customer_segment'].unique()

for segment in segments:
    segment_data = payment_behavior_credit_impact_df[
        payment_behavior_credit_impact_df['customer_segment'] == segment
    ]
    
    # Hovertext
    hover_credit = [
        f"<b>{segment}</b><br>{cat}<br>Credit Score: {val:.0f}"
        for cat, val in zip(segment_data['payment_behavior_category'], segment_data['avg_credit_score'])
    ]
    hover_util = [
        f"<b>{segment}</b><br>{cat}<br>Utilization: {val:.1f}%" if val is not None else f"<b>{segment}</b><br>{cat}<br>Utilization: N/A"
        for cat, val in zip(segment_data['payment_behavior_category'], segment_data['avg_credit_utilization_pct'])
    ]
    
    # Credit Score
    fig.add_trace(
        go.Bar(
            name=segment,
            x=segment_data['payment_behavior_category'],
            y=segment_data['avg_credit_score'],
            text=segment_data['avg_credit_score'],
            texttemplate='%{text:.0f}',
            textposition='outside',
            hovertext=hover_credit,
            hoverinfo="text"
        ),
        row=1, col=1
    )
    
    # Credit Utilization
    fig.add_trace(
        go.Bar(
            name=segment,
            x=segment_data['payment_behavior_category'],
            y=segment_data['avg_credit_utilization_pct'],
            hovertext=hover_util,
            hoverinfo="text",
            showlegend=False
        ),
        row=2, col=1
    )

# Axes
fig.update_xaxes(title_text="Payment Behavior", row=2, col=1)
fig.update_yaxes(title_text="Avg Credit Score", row=1, col=1)
fig.update_yaxes(title_text="Avg Utilization (%)", row=2, col=1)

# Layout
fig.update_layout(
    title="Causal Impact of Payment Behavior on Credit Health",
    height=800,
    barmode='group',
    plot_bgcolor='rgba(245,245,245,1)'
)

fig.show()

In [26]:
payment_behavior_credit_impact_df

,payment_behavior_category,credit_score_band,customer_segment,account_count,avg_payment_history_length,avg_late_payments,avg_days_late,avg_outstanding_balance,avg_late_fees_paid,avg_credit_utilization_pct,avg_credit_score,past_due_rate_pct,credit_score_effect_vs_ontime,utilization_effect_vs_ontime,additional_fees_vs_ontime,impact_classification,analyzed_at
0,Always On Time,Good,Affluent,34,8.6,0.00,0.0,182002.90,0.00,None,705.0,0.0,152.95,None,0.00,Baseline (Optimal Behavior),2025-12-27 07:58:05.790033+00:00
1,Always On Time,Fair,Affluent,47,7.9,0.00,0.0,194371.53,0.00,None,623.0,0.0,70.18,None,0.00,Baseline (Optimal Behavior),2025-12-27 07:58:05.790033+00:00
2,Always On Time,Poor,Affluent,141,7.8,0.00,0.0,196482.49,0.00,None,433.0,0.0,-119.71,None,0.00,Baseline (Optimal Behavior),2025-12-27 07:58:05.790033+00:00
3,Always On Time,Very Good,Affluent,38,7.2,0.00,0.0,187015.49,0.00,None,766.0,0.0,214.01,None,0.00,Baseline (Optimal Behavior),2025-12-27 07:58:05.790033+00:00
4,Always On Time,Excellent,Affluent,1,8.0,0.00,0.0,327989.74,0.00,None,800.0,0.0,247.54,None,0.00,Baseline (Optimal Behavior),2025-12-27 07:58:05.790033+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,Occasionally Late,Poor,Premium,425,15.0,1.89,1.3,194191.48,71.51,None,438.0,0.0,-115.24,None,71.51,Minor Credit Impact,2025-12-27 07:58:05.790033+00:00
69,Occasionally Late,Very Good,Premium,96,14.4,1.97,1.5,213614.76,73.00,None,769.0,0.0,215.67,None,73.00,Minor Credit Impact,2025-12-27 07:58:05.790033+00:00
70,Occasionally Late,Good,Premium,104,13.9,1.78,1.3,193872.52,67.47,None,707.0,0.0,153.69,None,67.47,Minor Credit Impact,2025-12-27 07:58:05.790033+00:00
71,Occasionally Late,Excellent,Premium,1,14.0,2.00,1.4,367637.17,95.95,None,800.0,0.0,246.34,None,95.95,Minor Credit Impact,2025-12-27 07:58:05.790033+00:00


### Payment Cost Heatmap

In [27]:
payment_cost_heatmap_query =  """
    SELECT payment_behavior_category, customer_segment,
        avg_late_fees_paid, credit_score_effect_vs_ontime
    FROM gold.analytics_payment_timeliness_impact
"""

payment_cost_heatmap_df = fetch_data(payment_cost_heatmap_query)

# Pivot table for late fees
pivot = payment_cost_heatmap_df.pivot_table(
    values='avg_late_fees_paid',
    index='payment_behavior_category',
    columns='customer_segment'
).sort_index(ascending=True)  # Optional: sort behaviors logically

# Optionally, create hover text combining fees and credit score effect
hover_text = payment_cost_heatmap_df.pivot_table(
    values='credit_score_effect_vs_ontime',
    index='payment_behavior_category',
    columns='customer_segment'
)

fig = go.Figure(data=go.Heatmap(
    z=pivot.values,
    x=pivot.columns,
    y=pivot.index,
    colorscale='Reds',
    text=pivot.values,
    texttemplate='$%{text:,.0f}',
    textfont={"size": 10},
    hovertemplate=(
        '<b>%{y}</b> - <b>%{x}</b><br>'
        'Avg Late Fees: $%{z:,.2f}<br>'
        'Credit Score Effect: %{customdata:.2f}<extra></extra>'
    ),
    customdata=hover_text.values,
    colorbar=dict(title="Late Fees ($)")
))

fig.update_layout(
    title="Average Late Fees Paid by Payment Behavior & Segment",
    xaxis_title="Customer Segment",
    yaxis_title="Payment Behavior",
    xaxis_tickangle=-45,
    height=500,
    margin=dict(l=100, r=50, t=100, b=150)
)

fig.show()

In [28]:
payment_cost_heatmap_df

,payment_behavior_category,customer_segment,avg_late_fees_paid,credit_score_effect_vs_ontime
0,Always On Time,Affluent,0.00,70.18
1,Always On Time,Affluent,0.00,152.95
2,Always On Time,Affluent,0.00,247.54
3,Always On Time,Affluent,0.00,214.01
4,Always On Time,Affluent,0.00,-119.71
...,...,...,...,...
68,Has Missed Payments,Premium,241.44,246.34
69,Has Missed Payments,Premium,193.66,-113.82
70,Has Missed Payments,Premium,186.46,70.70
71,Has Missed Payments,Premium,182.30,214.88
